In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Definer variabler der bruges til at filtrere data med.

In [2]:
NB = pd.to_datetime('01-05-2023')#indflytningsdato for NordBlok NB md-dd-yyyy

MA = 10 #Maksimal alder i år ved indflytningen

NBMA = NB.replace(year=NB.year - MA)#dato for ælste udstyr der kan flyttes i NordBlokken

BG = 5000 #BagatelGrænse for pris

filtAfd = False #Skal der filterers på afdelinger

EJER = 'Bispebjerg og Frederiksberg Hospital'
#EJER = 'Nordsjællands Hospital'
#print(NBMA)
FilterInput = pd.DataFrame({'Filter funktion':['Ejer', 'Indflytnings dato', 'Udstyrets minimum alder [år]','Nyeste dato [dato]', 'Bagatelgrænse i kr', 'Ekskluderede grupper'],
                           'Værdi': [EJER, NB, MA, NBMA, BG, 'nan']})

# Læs data
Først skal dataen fra Medusa læses ind, derefter skal data til filtereingen læses ind. Til slut tilpasses dataen inden filtereingen begynder.

In [3]:
#indlæs regneark med Meudsa udtræk.
filename = ['BBH.xlsx','FRH.xlsx'] #'BBH+FRH 19-03-20.xlsx' #filnavn på Meudusa udtræk i excel

filterIndput = 'filterIndput.xlsx' #Er en fil med input til filtrering, som er uafhængig af Medusa udtræk

AFD = 'Afdelinger.xlsx' #Hvis vi vil sortere afdelinger ud, kan gøres vha dette regneark.
filtAfd = False #Hvis der skal filteres på afdelinger, skal denne variable til True

df = pd.DataFrame()
for fil in filename:
    df = pd.read_excel(fil).append(df,sort=False) # Medusa udtræk
#df.info()

In [4]:
#indlæs regnearket med filter betingelser
gz = pd.read_excel(filterIndput, sheet_name='Gråzone') # Gråzone udstyr
xmd = pd.read_excel(filterIndput, sheet_name='Xmodel') # Ekskluderete modeller

In [5]:
#indlæs regneark hvor de afdelinger der skal med i kørslen er listet.
aFd = pd.read_excel(AFD)
#afd.head()

### Tilpasninger på data

In [6]:
#Tilpasninger til Medusadataen
df['Indkøbt']= pd.to_datetime(df.Indkøbt)#konverter til datetime som Pandas kan regne på

df["Pris"] = df[["Indkøbspris","Genansk.pris"]].max(axis=1)# find den største værdi og sæt det ind i ny kolonne "Pris"
#df.head()

In [7]:
#Ejer kolonnen består af to dele, derfor splittes det op i to kolonner
x = df['Ejer (overafdeling) 2'].str.split(";", n = 1, expand = True)
df["Overafdeling"] =x[0]
df["Ejer"] = x[1]     

In [8]:
#Label kolonnen skal laves og fyldes med default værdier
df['Label'] = 'Skal undersøges endividuelt'
#df.head()
#df.tail()

In [9]:
#Det næste laver en liste over afdelinger og skriver det i et regneark - så vi kan vælge afdelinger ud som vi vil se nærmere på
if 0: #behøver kun at køre én gang - ænder 0 til 1 
    Afdl = pd.unique(df['Ejer (overafdeling) 2'])
    afdl = pd.DataFrame(Afdl)
    with pd.ExcelWriter('Afdelinger.xlsx') as writer:
        afdl.to_excel(writer,index = False)

# Så begynder filtereringen

In [10]:
fdf = df.loc[df.Ejer.str.contains(EJER)== True] #filterer på Ejer
if filtAfd:
    fdf = fdf.loc[fdf.Overafdeling.isin(aFd['Afdeling'])] #filterer på afdeling
#fdf.head()

In [11]:
#fdf = fdf.loc[~fdf.Model.isin(xmd.Model),:] # fjern ekskluderede modeller
#ffdf = fdf.loc[fdf.Model.isin(xmd.Model),:] # find ekskluderede modeller
#print(len(ffdf))

In [12]:
#filter på kategori 1 = genbrug
filt1 = (fdf['Indkøbt'] > NBMA) & True #der kan sættes flere betingelser ind vha & eller |
fdf.loc[filt1, 'Label'] = 'Genbruges og flyttes'  
#fdf.head(10) #filtrer på alder

C:\Users\bkri0025\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
#filter på kategori 2 = budget
filt2= (fdf['Pris'] < BG) & True #der kan sættes flere betingelser ind vha & eller |
fdf.loc[filt2,'Label'] = 'Afdelingen har selv budget' 
fdf.head() #filtrer på pris

C:\Users\bkri0025\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,App.nr,Gruppenavn,Fabrikat,Model,Indkøbspris,Genansk.pris,Indkøbt,Ejer (overafdeling),Ejer (overafdeling) 2,Afdeling (placering),Afsnit (Placering),Pris,Overafdeling,Ejer,Label
0,19657,"Litotripsisystem, ultralyd",EMS,LithoClast Master,0.0,240000.0,2015-11-17,"Centraloperationsgang, FRH","Centraloperationsgang, FRH; Bispebjerg og Fred...","Urologisk afdeling U, FRH","Centraloperationsgang Urologi, FRH; Urologisk ...",240000.0,"Centraloperationsgang, FRH",Bispebjerg og Frederiksberg Hospital,Genbruges og flyttes
1,32084,"Termometer, øre, infrarød",Braun,ThermoScan Pro 4000,875.0,875.0,2013-07-18,"Medicinsk og Geriatrisk Overafdeling, FRH","Medicinsk og Geriatrisk Overafdeling, FRH ; Bi...","AMA., Akut modtageafdeling AKM","MMA3, Medicinsk Modtageafsnit 3 ; AMA Akutmodt...",875.0,"Medicinsk og Geriatrisk Overafdeling, FRH",Bispebjerg og Frederiksberg Hospital,Afdelingen har selv budget
3,32067,"Termometer, øre, infrarød",Braun,ThermoScan Pro 4000,875.0,875.0,2013-07-30,"Medicinsk og Geriatrisk Overafdeling, FRH","Medicinsk og Geriatrisk Overafdeling, FRH ; Bi...","AMA., Akut modtageafdeling AKM","MMA3, Medicinsk Modtageafsnit 3 ; AMA Akutmodt...",875.0,"Medicinsk og Geriatrisk Overafdeling, FRH",Bispebjerg og Frederiksberg Hospital,Afdelingen har selv budget
4,1087697,"Patientvægt, gulv, elektronisk",Tanita,WB-110,5000.0,5000.0,2013-07-31,"Medicinsk og Geriatrisk Overafdeling, FRH","Medicinsk og Geriatrisk Overafdeling, FRH ; Bi...","AMA., Akut modtageafdeling AKM","MMA3, Medicinsk Modtageafsnit 3 ; AMA Akutmodt...",5000.0,"Medicinsk og Geriatrisk Overafdeling, FRH",Bispebjerg og Frederiksberg Hospital,Genbruges og flyttes
5,32054,"Termometer, øre, infrarød",Braun,ThermoScan Pro 4000,875.0,875.0,2013-08-13,"Ortopædkirurgisk afdeling O, FRH","Ortopædkirurgisk afdeling O, FRH; Bispebjerg o...","Ortopædkirurgisk afdeling O, FRH","Ortopædkirurgisk Sengeafsnit, FRH; Ortopædkiru...",875.0,"Ortopædkirurgisk afdeling O, FRH",Bispebjerg og Frederiksberg Hospital,Afdelingen har selv budget


In [14]:
fdf = fdf.loc[~fdf.Gruppenavn.isin(gz.Gruppenavn)] #fjern gråzone udstyr

In [15]:
fdf = fdf.drop(['Indkøbspris', 'Genansk.pris', 'Ejer (overafdeling)'], axis=1)

In [16]:
#cols = fdf.columns.tolist()
#print(cols)

In [17]:
cols = ['App.nr', 'Gruppenavn', 'Fabrikat', 'Model', 'Label', 'Indkøbt', 'Pris', 'Overafdeling', 'Afdeling (placering)', 'Afsnit (Placering)', 'Ejer']
fdf = fdf[cols]

In [18]:
fdf = fdf.sort_values(by=['Gruppenavn','Indkøbt'])#sorter på gruppenavn og indkøbsdato
#fdf.info()

# Opdeling i afdelinger

In [19]:
Afd = pd.unique(fdf['Overafdeling'])
Afd.sort()

#print(Afd)

fdf['Overafdeling'] = fdf['Overafdeling'].str.slice(0,27)
afd = pd.unique(fdf['Overafdeling'])
afd.sort()

i=0
antal =[]
pris =[]
for x in afd:
    xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
    antal.append(len(xls)) # find antal udstyr i afdelingen
    pris.append(xls['Pris'].sum()) # find den samlede værdi af udstyret 
    i = i+len(xls)
   # print(x[0:28], "  ",len(xls))    
#print(i) #opsummetet antal
#antal
StatusArk =pd.DataFrame(list(zip(Afd, antal, pris)), columns =['Afdeling', 'Antal udstyr', 'Værdi'])
#StatusArk

In [20]:
Grupper = pd.DataFrame(fdf.groupby(['Overafdeling', 'Gruppenavn'])['Gruppenavn'].count())
#Grupper.unstack()

In [21]:
with pd.ExcelWriter('Udstyr med labeler opdelt.xlsx') as writer:
    FilterInput.to_excel(writer, sheet_name='Opsumering',index = False)
    gz.to_excel(writer, sheet_name='Opsumering',startrow = 6,startcol = 1,index = False)
    StatusArk.to_excel(writer, sheet_name='Opsumering',startcol = 4, index = False)
    Grupper.to_excel(writer, sheet_name='Udstyrstyper i afdelingerne',index = True)
    Grupper.unstack().to_excel(writer, sheet_name='Udstyrstyper i afdelingerne', startcol = 5,index = True)
    
    for x in afd:
        xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
        if len(xls)>0:
            xls.to_excel(writer, sheet_name=x[0:27],index = False)

# Skriv det hele i et regneark

In [22]:
with pd.ExcelWriter('Udstyr med labeler.xlsx') as writer:
    
    fdf.to_excel(writer,index = False)